In [1]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 2.1.0
Default GPU Device: /device:GPU:0


In [2]:
!git clone https://github.com/DavidReveloLuna/keras-retinanet.git

Cloning into 'keras-retinanet'...


In [3]:
cd keras-retinanet/

C:\Users\Alienware\Pictures\prueba5\keras-retinanet


In [4]:
!pip install .
!python setup.py build_ext --inplace

Processing c:\users\alienware\pictures\prueba5\keras-retinanet
  Created wheel for keras-retinanet: filename=keras_retinanet-0.5.1-cp37-cp37m-win_amd64.whl size=131671 sha256=92c47973f31535ebc1b1a3125f5460b16df65c85c6e86774ac82ab677668059e
  Stored in directory: c:\users\alienware\appdata\local\pip\cache\wheels\31\b5\62\5222480c5e299d379ee007ec466b711d5e1cd00223a996d7a0
Successfully built keras-retinanet
  Attempting uninstall: keras-retinanet
    Found existing installation: keras-retinanet 0.5.1
    Uninstalling keras-retinanet-0.5.1:
      Successfully uninstalled keras-retinanet-0.5.1


ERROR: Could not install packages due to an EnvironmentError: [WinError 5] Acceso denegado: 'c:\\users\\alienware\\anaconda3\\envs\\entornogpu\\lib\\site-packages\\~eras_retinanet\\utils\\compute_overlap.cp37-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



running build_ext
skipping 'keras_retinanet/utils\compute_overlap.c' Cython extension (up-to-date)
copying build\lib.win-amd64-3.7\keras_retinanet\utils\compute_overlap.cp37-win_amd64.pyd -> keras_retinanet\utils


In [6]:
cd C:\Users\Alienware\Pictures\prueba5

C:\Users\Alienware\Pictures\prueba5


In [7]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

In [8]:
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

In [9]:
from keras.models import load_model
from keras_retinanet import models

model_path = os.path.join('snapshots', sorted(os.listdir('snapshots/'), reverse=True)[0])
print(model_path)

model = models.load_model(model_path, backbone_name='resnet50')
model = models.convert_model(model)

labels_to_names = pd.read_csv('classes.csv', header=None).T.loc[0].to_dict()

snapshots\resnet50_csv_50.h5
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-22.627417, -11.313708,  22.627417,  11.313708],
       [-28.50876 , -14.25438 ,  28.50876 ,  14.25438 ],
       [-35.918785, -17.959393,  35.918785,  17.959393],
       [-16.      , -16.      ,  16.      ,  16.      ],
       [-20.158737, -20.158737,  20.158737,  20.158737],
       [-25.398417, -25.398417,  25.398417,  25.398417],
       [-11.313708, -22.627417,  11.313708,  22.627417],
       [-14.25438 , -28.50876 ,  14.25438 ,  28.50876 ],
       [-17.959393, -35.918785,  17.959393,  35.918785]], dtype=float32)> anchors
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32, numpy=
array([[-45.254833, -22.627417,  45.254833,  22.627417],
       [-57.01752 , -28.50876 ,  57.01752 ,  28.50876 ],
       [-71.83757 , -35.918785,  71.83757 ,  35.918785],
       [-32.      , -32.      ,  32.      ,  32.      ],
       [-40.317474, -40.317474,  40.317474,  40.317474],
       [

In [10]:
# Función que realizaz la predicción

import skimage.io as io

def predict(image):
    image = preprocess_image(image.copy())
    image, scale = resize_image(image)

    boxes, scores, labels = model.predict_on_batch(
     np.expand_dims(image, axis=0)
     )

    boxes /= scale

    return boxes, scores, labels

In [14]:
# Captura de Video
camera = cv2.VideoCapture(0)
camera_height = 500

while(True):

    _, frame = camera.read()
    

    frame = cv2.flip(frame, 1)

    boxes, scores, labels = predict(frame)

    draw = frame.copy()


    for box, score, label in zip(boxes[0], scores[0], labels[0]):
        if score > 0.8:
            print(box)
            b = box.astype(int)
            color = label_color(label)
            draw_box(draw, b, color=color)
            caption = "{} {:.3f}".format(labels_to_names[label], score)
            draw_caption(draw, b, caption)
            

    # show the frame
    cv2.imshow("Test out", draw)


    key = cv2.waitKey(1)

    # quit camera if 'q' key is pressed
    if key & 0xFF == ord("q"):
        break

camera.release()
cv2.destroyAllWindows()

[ 36.13889 132.17258 495.8289  479.40002]
[118.618256 115.61525  452.2146   394.08844 ]
[ 99.55673 128.99641 496.675   392.23206]
[129.74901 124.96983 460.3914  393.72607]
[108.89389 121.99285 424.30753 391.983  ]
[  0.      151.88992 483.35303 384.6057 ]
[  0.      142.17976 489.45554 390.78964]
[  0.      149.74771 478.4426  399.94073]
[  0.      137.43796 476.79993 407.64822]
[  0.      142.54727 477.36597 408.84625]
[ 35.334557  52.220207 396.19327  357.97635 ]
[  8.6653   53.39585 394.73154 378.71768]
[  0.        54.920765 396.94244  370.98715 ]
[  0.        62.828766 389.5462   382.1255  ]
[105.701904 134.72804  394.09012  479.40002 ]
[108.46884 150.44167 402.26776 479.40002]
[ 92.30651  84.40504 380.92026 479.40002]
[ 16.001312  48.21035  479.89227  399.14716 ]
[ 28.700722  50.116142 561.10474  434.2213  ]
[ 37.937805  48.77894  560.541    431.56628 ]
[ 25.450773  49.187096 546.24225  437.8617  ]
[ 20.148249  43.56621  537.0908   427.57928 ]
